In [36]:
%matplotlib inline
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np
import aruco

In [37]:
# Globals
CV_CAP_PROP_POS_MSEC = 0
CV_CAP_PROP_FRAME_COUNT = 7

def dist(x1,y1,x2,y2):
    return ((x2-x1)**2 + (y2-y1)**2)**0.5

In [38]:
# set up - choose files
stripped_raw_data_file_path = "board_stripped.txt"
visual_intake_times_path = "visualIntakeTimes.txt"
video_file_path = "AlisonBoard1-1-recording.avi"

In [39]:
# create detector
detector = aruco.MarkerDetector()

In [40]:
# load eye tracking data
eye_data = []
data_file_object  = open(stripped_raw_data_file_path, "r")
data_file_object.readline()
for line in data_file_object.readlines():
    eye_data.append(map(float, line.split()))
data_file_object.close()

In [41]:
# load visual intake times
vi_times = []
data_file_object = open(visual_intake_times_path, "r")
for line in data_file_object.readlines():
    vi_times.append(float(line))
data_file_object.close()

In [42]:
# load video

cap = cv2.VideoCapture(video_file_path)
ret, frame = cap.read()
    
if not ret:
    print("can't open video!")
    sys.exit(-1)

vi_number = 0
time = vi_times[vi_number]

markers = detector.detect(frame)
lastClosestMarker = markers[0].id
regressions = []

while ret and int(time/16.3) < len(eye_data):
    markers = detector.detect(frame)

    if len(markers) == 0:
        # read next frame
        ret, frame = cap.read()
        vi_number += 1
        time = vi_times[vi_number]
        continue
    
    # variables for accessing marker coordinates
    upperLeft = 0 #corner of marker
    upperRight = 1
    lowerLeft = 3
    lowerRight = 2
    x = 0
    y = 1
    
    closestMarker = [markers[0].id, markers[0][upperLeft][x], markers[0][upperLeft][y]]
    gaze_pos_x = eye_data[int(time/16.3)][1]
    gaze_pos_y = eye_data[int(time/16.3)][2]
    
    for marker in markers:
        # get marker ID and point positions
        distance_curr_marker = dist(gaze_pos_x, gaze_pos_y, marker[upperLeft][x], marker[upperLeft][y])
        distance_closest_marker = dist(gaze_pos_x, gaze_pos_y, closestMarker[1], closestMarker[2])
        if distance_curr_marker < distance_closest_marker:
            closestMarker = [marker.id, marker[upperLeft][x], marker[upperLeft][y]]
    
    if closestMarker[0] < lastClosestMarker-15:
        regressions.append([time, lastClosestMarker, closestMarker[0]])
    
    lastClosestMarker = closestMarker[0]
    
    # read next frame
    ret, frame = cap.read()
    vi_number += 1
    time = vi_times[vi_number]
    
    

In [43]:
print len(regressions)

37


In [44]:
for regression in regressions:
    print round(regression[0]/1000, 3), "\t", regression[1], "\t", regression[2]

0.799 	49 	20
1.331 	48 	4
3.295 	33 	1
4.676 	49 	1
5.941 	48 	31
7.289 	33 	17
8.603 	48 	17
12.181 	48 	17
16.857 	49 	33
20.002 	47 	31
23.629 	49 	33
32.515 	49 	33
37.274 	50 	33
37.806 	65 	21
39.937 	65 	49
48.423 	66 	37
53.365 	51 	23
53.748 	66 	49
64.913 	50 	21
67.576 	51 	35
68.841 	67 	51
79.074 	68 	52
79.64 	68 	20
82.968 	68 	52
85.148 	68 	38
85.93 	68 	52
89.591 	53 	37
91.388 	53 	21
107.313 	69 	52
120.125 	64 	19
128.945 	64 	48
131.258 	64 	47
134.852 	64 	21
141.109 	65 	49
146.4 	65 	36
150.011 	65 	49
154.521 	65 	33
